In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import joblib

empresa = {
    'Apple': 'AAPL',
    'Microsoft': 'MSFT',
    'Google': 'GOOG',
    'Amazon': 'AMZN',
    'Tesla': 'TSLA',
    'NVIDIA': 'NVDA',
    'Meta': 'META'
}

def reg_coef(line, df_reg, p):
    i = 1
    f = i + p
    reg = LinearRegression()
    y = np.array(df_reg.loc[line][2:f].to_list())
    X = np.array([x for x in range(1,len(y)+1)]).reshape((-1, 1))

    
    reg.fit(X,y)
    return reg.coef_[0]

def feature_engineering(empresa):
    df_list = []
    for name,ativo in empresa.items():
        df = pd.read_csv(f'../Dataset/Refined/{name.lower()}_ref.csv')
        df['Date'] = pd.to_datetime(df['Date'])

        df_temp = df[['Date','Close']].copy()

        for i in range(1,30+1):
            df_temp['lag_'+str(i)] = df_temp['Close'].shift(i)

        df_temp.dropna(how='any',inplace=True)
        df_temp.reset_index(drop=True,inplace=True)
        
        coef_list_3 = []
        coef_list_7 = []
        coef_list_10 = []
        coef_list_15 = []
        coef_list_20 = []
        coef_list_30 = []
        for line in df_temp.index:
            coef_list_3.append(reg_coef(line=line, df_reg =df_temp, p=3))
            coef_list_7.append(reg_coef(line=line, df_reg =df_temp, p=7))
            coef_list_10.append(reg_coef(line=line, df_reg =df_temp, p=10))
            coef_list_15.append(reg_coef(line=line, df_reg =df_temp, p=15))
            coef_list_20.append(reg_coef(line=line, df_reg =df_temp, p=20))
            coef_list_30.append(reg_coef(line=line, df_reg =df_temp, p=30))

        df_temp['coef_3'] = coef_list_3
        df_temp['coef_7'] = coef_list_7
        df_temp['coef_10'] = coef_list_10
        df_temp['coef_15'] = coef_list_15
        df_temp['coef_20'] = coef_list_20
        df_temp['coef_30'] = coef_list_30

        final_temp = df.merge(df_temp[['Date','coef_3','coef_7','coef_10','coef_15','coef_20','coef_30']],how='left',on='Date').dropna(subset=['coef_30']).reset_index(drop=True).copy()
        final_temp.to_csv(f'../Dataset/Main/{name.lower()}_main.csv',index=False)
        df_list.append(final_temp)

    new_df = pd.concat(df_list)
    new_df.reset_index(drop=True,inplace=True)
    new_df.to_csv(f'../Dataset/Main/full_main.csv',index=False)

feature_engineering(empresa)

new_df =  pd.read_csv(f'../Dataset/Main/full_main.csv')


In [2]:
import pandas as pd

df = pd.read_csv(f'../Dataset/Refined/apple_ref.csv')

df

,Date,Close,Mean,Dif,Volume,Empresa,Symbol,target,moment,Action
0,2000-01-03,0.848323,NaN,-0.053523,535796800,Apple,AAPL,0,hold,0
1,2000-01-04,0.776801,NaN,0.043576,512377600,Apple,AAPL,0,hold,0
2,2000-01-05,0.788168,0.804431,-0.001895,778321600,Apple,AAPL,0,hold,0
3,2000-01-06,0.719961,0.761643,0.084312,767972800,Apple,AAPL,0,hold,0
4,2000-01-07,0.754065,0.754065,-0.022736,460734400,Apple,AAPL,0,hold,0
...,...,...,...,...,...,...,...,...,...,...
5926,2023-07-25,193.358887,192.510040,-0.289602,37283200,Apple,AAPL,0,hold,0
5927,2023-07-26,194.237701,193.362218,-0.828883,47471900,Apple,AAPL,0,hold,0
5928,2023-07-27,192.959427,193.518672,2.796227,47460200,Apple,AAPL,0,hold,0
5929,2023-07-28,195.565918,194.254349,-1.158439,48291400,Apple,AAPL,1,sell,1
